In [1]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import pickle
import glob
import time

In [2]:
piece_dim = 8
key_dim = 12
depth = 32
gamma = 0.99
lam = 0.95
temperature = 1.0

In [3]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
# pretrainer = Pretrainer(gamma=lam)

In [4]:
# players_data = pretrainer._load_data()

In [5]:
# pretrainer._load_dset(players_data)

In [6]:
# max_len = pretrainer._max_len
max_len = 10

In [7]:
# gt_dset = pretrainer._cache_dset()

In [8]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len,
                    out_dim=key_dim)

In [9]:
agent_optimizer = keras.optimizers.Adam(1e-4)
agent.compile(optimizer=agent_optimizer)

In [10]:
logits, piece_scores, key_scores = agent((tf.random.uniform((32, 28, 10, 1)),
                                          tf.random.uniform((32, 7), minval=0, maxval=8, dtype=tf.int32),
                                          tf.random.uniform((32, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
agent.summary(), tf.shape(logits), tf.shape(piece_scores), tf.shape(key_scores)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 70, 32)              37312     
                                                                 
 seq_embedding (SeqEmbedding  multiple                 256       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 384       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_2 (DecoderLayer)  multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([32, 10, 12])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4, 10,  7])>)

In [11]:
critic = TetrisModel(piece_dim=piece_dim,
                     key_dim=key_dim,
                     depth=depth,
                     num_heads=4,
                     num_layers=4,
                     max_length=max_len,
                     out_dim=1)

In [12]:
critic_optimizer = keras.optimizers.Adam(1e-4)
critic.compile(optimizer=critic_optimizer)

In [13]:
values, piece_scores, key_scores = critic((tf.random.uniform((32, 28, 10, 1)),
                                           tf.random.uniform((32, 7), minval=0, maxval=8, dtype=tf.int32),
                                           tf.random.uniform((32, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
critic.summary(), tf.shape(values), tf.shape(piece_scores), tf.shape(key_scores)

Model: "tetris_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_9 (Sequential)   (32, 70, 32)              37312     
                                                                 
 seq_embedding_2 (SeqEmbeddi  multiple                 256       
 ng)                                                             
                                                                 
 seq_embedding_3 (SeqEmbeddi  multiple                 384       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  37984     
                                                                 
 piece_dec_2 (DecoderLayer)  multiple               

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([32, 10,  1])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4, 32,  4, 10,  7])>)

In [14]:
agent_checkpoint = tf.train.Checkpoint(model=agent, optim=agent.optimizer)
agent_checkpoint.restore('agent_checkpoint/agent_pretrained-1')

In [15]:
critic_checkpoint = tf.train.Checkpoint(model=critic, optim=critic.optimizer)
critic_checkpoint.restore('critic_checkpoint/critic_pretrained-1')

In [16]:
# agent.load_weights('agent_weights_small/agent_finetuned_3074')
# critic.load_weights('critic_weights_small/critic_finetuned_3074')

# with open('agent_weights_small/optimizer_3074.pkl', 'rb') as f:
#     weight_values = pickle.load(f)
# agent.optimizer.set_weights(weight_values)

# with open('critic_weights_small/optimizer_3074.pkl', 'rb') as f:
#     weight_values = pickle.load(f)
# critic.optimizer.set_weights(weight_values)

In [17]:
ref_agent = TetrisModel(piece_dim=piece_dim,
                        key_dim=key_dim,
                        depth=depth,
                        num_heads=4,
                        num_layers=4,
                        max_length=max_len,
                        out_dim=key_dim)

In [18]:
logits, piece_scores, key_scores = ref_agent((tf.random.uniform((1, 28, 10, 1)),
                                              tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                                              tf.random.uniform((1, max_len), minval=0, maxval=key_dim, dtype=tf.int32)), return_scores=True)
tf.shape(logits), tf.shape(piece_scores), tf.shape(key_scores)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 10, 12])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4,  1,  4,  7, 70])>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 4,  1,  4, 10,  7])>)

In [19]:
ref_checkpoint = tf.train.Checkpoint(model=ref_agent)
ref_checkpoint.restore('agent_checkpoint/agent_pretrained-1')

In [20]:
# epochs = 10

In [21]:
# actor_losses, critic_losses, accs = pretrainer.train(agent, critic, gt_dset, epochs)

In [22]:
%matplotlib qt

In [23]:
# plt.plot(actor_losses)
# plt.plot(critic_losses)
# plt.plot(accs)

In [24]:
trainer = Trainer(agent=agent,
                  critic=critic,
                  ref_model=ref_agent,
                  max_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  temperature=temperature,
                  max_episode_steps=100,
                  buffer_cap=1000)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.fill_replay_buffer()

Current Episode: 16

In [ ]:
while True:
    trainer.train(gens=100, train_steps=10, training_actor=True)
    agent.save_weights(f'agent_weights_small/agent_finetuned_{trainer.wandb_run.step}')
    critic.save_weights(f'critic_weights_small/critic_finetuned_{trainer.wandb_run.step}')

In [36]:
if 'y' in input('YOU SURE?????'):
    agent.save_weights(f'agent_weights_small/agent_finetuned_{trainer.wandb_run.step}')
    critic.save_weights(f'critic_weights_small/critic_finetuned_{trainer.wandb_run.step}')

symbolic_weights = getattr(agent.optimizer, 'weights')
weight_values = K.batch_get_value(symbolic_weights)
with open(f'agent_weights_small/optimizer_{trainer.wandb_run.step}.pkl', 'wb') as f:
    pickle.dump(weight_values, f)

symbolic_weights = getattr(critic.optimizer, 'weights')
weight_values = K.batch_get_value(symbolic_weights)
with open(f'critic_weights_small/optimizer_{trainer.wandb_run.step}.pkl', 'wb') as f:
    pickle.dump(weight_values, f)

trainer.wandb_run.step

YOU SURE????? y


3074

In [32]:
episode_data = trainer.player.run_episode(agent, critic, max_steps=100, greedy=True, renderer=trainer.renderer)

In [33]:
episode_boards, episode_pieces, episode_inputs, episode_probs, episode_values, episode_rewards = episode_data

In [34]:
episode_advantages, episode_returns = trainer._compute_gae(episode_values, episode_rewards, trainer.gamma, trainer.lam)

In [35]:
fig, ax = plt.subplots()
ax.plot(episode_returns, label='Returns')
ax.plot(episode_values, label='Values')
ax.legend()
tf.reduce_sum(episode_rewards)

<tf.Tensor: shape=(), dtype=float32, numpy=4.25>

In [53]:
fig, ax = plt.subplots()
ax.plot(episode_rewards, label='Rewards')
ax.plot(episode_advantages, label='Advantages')
ax.legend()

In [33]:
trainer.wandb_run.finish()

critic_loss,▄▂▁▁▁▃█▅▂▂▂▁▃▃▂▄▄▄▄▃▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂
entropy,▄▁▂▆▄▆▅▇▃▅▆▄▆▆▆█▆▅▇▅▃▅▃▅▆▃█▅▆▅▆▅▇▃█▅▇▇█▆
kl_div,▅▅▄▂▃▂▂▄▁▄▄▂▂▁▄▅▂▄▄▁▄▂▄▄▃█▄▃▂▇▃▆▅▃▆▆▃▂▃▄
ppo_loss,▆▅▂▇▁▅▇▂▂▄▅▃▅▅▃▆▄▇▇▅▂▅▇▇▂█▄▇▃▃▅▁▃▄▆▄█▆▂▃
reward,▄▅▆▄▇▄▄▄▆▅▄▄▅▅▅▅▃▃▆▅▄▅▅▂▃▅▇▅▅▃▃▂▁▇▄▅▅▄▅█
reward_per_piece,▃▄▅▄▃▄▆▂▄▃▆▄▂▅▄▁▂▃▅▃▅▅▆▄▄▆▄█▃▃▆▅▄▄▂▃▄▃▂▅
unclipped_proportion,▅▅▄▄▆▆▇▅▆▄█▆▄█▅▇█▆▃▅▅▃▆▃▆▅▅▄▅▆▃▁▄▅▃▅▆▄▆▄
critic_loss,0.13156
entropy,-0.2607
kl_div,0.20674
ppo_loss,0.00448
